---

# Анализ данных в Pandas

---

## 1 | Немного о природе данных

---

Импортируем библиотеки и загрузим данные:

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/content/sample_data/avito_flats.csv', sep=';', index_col=0)
print(df.columns)

Index(['id', 'Город', 'Район', 'Улица', 'Геометка_дома', 'Тип_сделки',
       'Срок_аренды', 'Цена', 'Тип_дома', 'Количество_комнат', 'Общая_площадь',
       'Количество_этажей', 'Материал_стен', 'Тип_продажи', 'Дата',
       'Площадь_кухни', 'Жилая_площадь', 'Этаж'],
      dtype='object')


---

### 1.2 | Описательные статистики

Узнаем среднюю площадь квартир по количеству комнат с кирпичными стенами:

In [3]:
df[df['Материал_стен'] == 'кирпичный'] \
    .groupby('Количество_комнат') \
    .agg({'Общая_площадь': 'mean'})

,Общая_площадь
Количество_комнат,
1,38.907625
2,61.601440
3,93.072085
4,120.500000
5,230.100000
своб. планировка,58.000000
студия,26.714849


---

### 1.3 | Предобработка данных

---

Заменим все ячейки со значением 'Не указано' на $\mathsf{np.nan}$, преобразуем такие строки с ценой, как '9 616 960' к '9616960', а также преобразуем все численные данные к типу $\mathsf{np.float64}$:

In [4]:
df = df.replace('Не указано', np.nan)
df['Цена'] = df['Цена'].str.replace(' ', '')

df = df.astype({'Цена': np.float64, 'Общая_площадь': np.float64,
                'Площадь_кухни': np.float64, 'Жилая_площадь': np.float64})
df['Жилая_площадь'].head()

0    12.2
1    27.0
2     NaN
3    24.4
4    37.7
Name: Жилая_площадь, dtype: float64

---

Рассмотрим уникальные значения некоторых колонок, чтобы отсеять бесполезные признаки:

In [5]:
df['Тип_сделки'].unique()

array(['Продажа'], dtype=object)

In [6]:
df['Срок_аренды'].unique()

array(['Не аренда'], dtype=object)

In [7]:
df['Тип_дома'].unique()

array(['Новостройка'], dtype=object)

In [8]:
df['Количество_этажей'].unique()

array([nan])

In [9]:
df['Тип_продажи'].unique()

array([nan])

In [10]:
df['Район'].value_counts(dropna=False)

NaN                     6616
Приморский район           2
Красносельский район       1
Name: Район, dtype: int64

In [11]:
df = df.drop(['Тип_сделки', 'Срок_аренды', 'Тип_дома',
              'Количество_этажей', 'Тип_продажи', 'Район'], axis=1)
df.columns

Index(['id', 'Город', 'Улица', 'Геометка_дома', 'Цена', 'Количество_комнат',
       'Общая_площадь', 'Материал_стен', 'Дата', 'Площадь_кухни',
       'Жилая_площадь', 'Этаж'],
      dtype='object')

---

Также могут встречаться и те признаки, у которых не практически все значения пропущены. Например, такие, как 'Жилая_площадь' и 'Площадь кухни':

In [12]:
df['Жилая_площадь'].size, df['Жилая_площадь'].isna().sum()

(6619, 1904)

In [13]:
df['Площадь_кухни'].size, df['Площадь_кухни'].isna().sum()

(6619, 2545)

Такие колонки можно и выкинуть, и наоборот — заполнить пропущенные места какими-то значениями. Давайте для разнообразия заполним пропущенные значения медианами:

In [14]:
df['Жилая_площадь'] = df['Жилая_площадь'].fillna(df['Жилая_площадь'].median())
df['Площадь_кухни'] = df['Площадь_кухни'].fillna(df['Площадь_кухни'].median())